## Step 1: Install Dependencies for SQL Database

In [1]:
%pip install flask flask-restful flask-sqlalchemy mysql-connector-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 7.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.3/133.3 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 7.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.3/227.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 505.5/505.5 kB 7.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 625.7/625.7 kB 7.8 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


## Step 2:Run Code


In [6]:
from flask import Flask, request, jsonify
import mysql.connector

app = Flask(__name__)

# Connect to the database
cnx = mysql.connector.connect(
    user='username',
    password='password',
    host='localhost',
    database='itanong'
)

# Get the duty rate for a given product_id
def get_tariff_rate_by_id(product_id):
    cursor = cnx.cursor()
    query = "SELECT product_description, rate_MFN, rate_ATIGA, effective_year FROM products WHERE product_id = %s"
    cursor.execute(query, (product_id,))
    result = cursor.fetchone()
    if result:
        return {
#             'product_id': int(product_id),
#             'product_description': result[0],
            'duty_rate_MFN': result[1],
            'duty_rate_ATIGA': result[2],
            'rate_effective_year': result[3]
        }
    else:
        return None

# Get the duty rate for a given product_description
def get_tariff_rate_by_description(product_description):
    cursor = cnx.cursor()
    query = "SELECT product_id, rate_MFN, rate_ATIGA, effective_year FROM products WHERE product_description = %s"
    cursor.execute(query, (product_description,))
    result = cursor.fetchone()
    if result:
        return {
#             'product_id': result[0],
#             'product_description': product_description,
            'duty_rate_MFN': result[1],
            'duty_rate_ATIGA': result[2],
            'rate_effective_year': result[3]
        }
    else:
        return None

# Get all categories
@app.route('/categories')
def get_categories():
    cursor = cnx.cursor()
#     query = "SELECT category_id, category_description FROM category"
    query = "SELECT category_description FROM category"
    cursor.execute(query)
    results = cursor.fetchall()
    categories = []
    for result in results:
        categories.append({
#             'category_id': result[0],
            'category_name': result[0]
        })
    return categories

# Get all products
@app.route('/products', methods=['GET'])
def get_products():
    category_id = request.args.get('category_id')
    category_name = request.args.get('category_name')
    if category_id:
        return get_products_by_category_id(category_id)
    elif category_name:
        return get_products_by_category_name(category_name)
    else:
        return get_all_products()

# Get all products
def get_all_products():
    cursor = cnx.cursor()
#     query = "SELECT product_id, product_description, category_id FROM products"
    query = "SELECT product_description FROM products"
    cursor.execute(query)
    results = cursor.fetchall()
    products = []
    for result in results:
        products.append({
#             'product_id': result[0],
            'product_name': result[0],
#             'category_name': get_category_name(result[2])
        })
    return products

# Get products within a specific category using category_id
def get_products_by_category_id(category_id):
    cursor = cnx.cursor()
    query = "SELECT p.product_description FROM products p JOIN category c ON p.category_id = c.category_id WHERE c.category_id = %s"
    cursor.execute(query, (category_id,))
    results = cursor.fetchall()
    products = []
    for result in results:
        products.append({
            'product_name': result[0],
        })
    return products

# Get products within a specific category using category_name
def get_products_by_category_name(category_name):
    cursor = cnx.cursor()
    query = "SELECT p.product_description FROM products p JOIN category c ON p.category_id = c.category_id WHERE c.category_description = %s"
    cursor.execute(query, (category_name,))
    results = cursor.fetchall()
    products = []
    for result in results:
        products.append({
            'product_name': result[0],
        })
    return products

@app.route('/tariff-rate', methods=['GET'])
def tariff_rate():
    product_id = request.args.get('product_id')
    product_description = request.args.get('product_description')
    if product_id:
        return get_tariff_rate_by_id(product_id)
    elif product_description:
        return get_tariff_rate_by_description(product_description)
    else:
        return "Missing arguments", 400


In [7]:
app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [03/Jun/2024 02:28:28] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [03/Jun/2024 02:28:29] "GET /favicon.ico HTTP/1.1" 404 -
